In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [2]:
data = pd.read_csv('Sentiment.csv')
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [3]:
#keep the necessary columns only
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [4]:
#function to remove unwanted characters
def pre_process_data(text):
    text = text.lower()
    new_text = re.sub('[^a-zA-z0-9\s]','',text)
    new_text = re.sub('rt', '', new_text)
    return new_text

data['text'] = data['text'].apply(pre_process_data)
    

In [5]:
data.head()

,text,sentiment
0,nancyleegrahn how did everyone feel about the...,Neutral
1,scottwalker didnt catch the full gopdebate la...,Positive
2,tjmshow no mention of tamir rice and the gopd...,Neutral
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive


In [6]:
#use Tensorflow’s tokenizer to tokenize our dataset, and Tensorflow’s pad_sequences to pad our sequences.

max_features = 2000
tokenizer = Tokenizer(num_words = max_features, split = ' ')
tokenizer.fit_on_texts(data['text'].values)
x = tokenizer.texts_to_sequences(data['text'].values)
x = pad_sequences(x,28)

y = pd.get_dummies(data['sentiment']).values

### Train and test

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.20)

###### Creating a deep learning model using an embedding layer and some LSTM

In [8]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length =x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout = 0.3, recurrent_dropout = 0.2, return_sequences = True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [10]:
batch_size = 512

model.fit(x_train, y_train, epochs = 10, batch_size=batch_size, validation_data=(x_test, y_test))


Train on 11096 samples, validate on 2775 samples
Epoch 1/10
11096/11096 [==============================] - 21s 2ms/sample - loss: 0.9640 - accuracy: 0.5970 - val_loss: 0.8897 - val_accuracy: 0.6076
Epoch 2/10
11096/11096 [==============================] - 16s 1ms/sample - loss: 0.8513 - accuracy: 0.6212 - val_loss: 0.8086 - val_accuracy: 0.6314
Epoch 3/10
11096/11096 [==============================] - 16s 1ms/sample - loss: 0.7635 - accuracy: 0.6684 - val_loss: 0.7616 - val_accuracy: 0.6695
Epoch 4/10
11096/11096 [==============================] - 17s 1ms/sample - loss: 0.6989 - accuracy: 0.6957 - val_loss: 0.7316 - val_accuracy: 0.6897
Epoch 5/10
11096/11096 [==============================] - 17s 1ms/sample - loss: 0.6601 - accuracy: 0.7176 - val_loss: 0.7296 - val_accuracy: 0.6923
Epoch 6/10
11096/11096 [==============================] - 17s 2ms/sample - loss: 0.6248 - accuracy: 0.7337 - val_loss: 0.7237 - val_accuracy: 0.6919
Epoch 7/10
11096/11096 [==============================] -

In [11]:
model.save('sentiment.h5')